# The Knapsack problem

#### Category: Integer programming (IP)

#### What is it about?
- Use integer programming to solve a basic combinatorial problem.

## Introduction

The 0/1 knapsack problem or rucksack problem is a problem in combinatorial optimization: Given a set of items, each with a weight and a value, determine which items to include in a collection so that the total weight is less than or equal to a given limit of 1400 gramms and the total value is as large as possible. It derives its name from the problem faced by someone who is constrained by a fixed-size knapsack and must fill it with the most valuable items. - adapted from <a href="https://en.wikipedia.org/wiki/Knapsack_problem" target="_blank">Wikipedia</a>

|   Item            | Value [$]    | Weight [g] |
|:------------------|:------------:|:----------:|
|   Hammer          | 8            | 500        |
|   Wrench          | 3            | 700        |
|   Screwdriver     | 6            | 400        |
|   Towel           | 11           | 300        |

## Mathematical model

#### Description
$
\begin{equation*}
I : \text{Set of items}\\
X_i : \text{Binary variable whether an item i should be packed}\\
v : \text{Values of items in \$}\\
w : \text{Weights of items in g}\\
w_{max} : \text{Maximum total weight in g}\\
\end{equation*}
$

#### Index set
$
\begin{equation*}
I = \{hammer, wrench, screwdriver, towel\}\\
\end{equation*}
$

#### Decision variables
$
\begin{equation*}
X_i \qquad i \in I, \: X \in \{0,1\}\\
\end{equation*}
$

#### Objective
$
\begin{equation*}
MAX \: \sum\limits_{i \in I}v_iX_i\\
\end{equation*}
$

#### Constraint: Maximum weight
$
\begin{equation*}
\sum\limits_{i \in I}w_iX_i \leq w_{max}\\
\end{equation*}
$

<hr>

## Pyomo implementation

#### Important: Because the following code cells build on each other, you MUST run every code cell starting from now! If you get an error, try selecting the cell and click "Cell" -> "Run All Above" in the taskbar above and then run the cell again.

#### Suggested workflow
1. Load all needed packages and data in your script and transform the data into a suitable structure.
2. Create a model object.
3. Define the index sets.
4. Based on the index sets, define the decision variables.
5. Specify the objective.
6. Specify the constraints.
7. Decide on a suitable solver depending on your problem and solve it.
8. Process the results.

### Step 1: Load all needed packages and data in your script and transform the data into a suitable structure

In [ ]:
from pyomo.environ import *

Specify the path to the solver executable:

In [ ]:
# For windows: r'../_Solvers/Cbc-2.9.9-win32-msvc14/bin/cbc.exe'
# For ubuntu bionic beaver: r'../_Solvers/Ubuntu_Bionic/Cbc-2.9.8/bin/cbc'
solver_path = r'../_Solvers/Cbc-2.9.9-win32-msvc14/bin/cbc.exe'

We use dictionaries to hold the values and weights of our items:

In [ ]:
v = {'hammer':8, 'wrench':3, 'screwdriver':6, 'towel':11}
w = {'hammer':500, 'wrench':700, 'screwdriver':400, 'towel':300}
w_max = 1400

### Step 2: Create a model object

In [ ]:
mo = ConcreteModel()

### Step 3: Define the index set
$
\begin{equation*}
I = \{hammer, wrench, screwdriver, towel\}\\
\end{equation*}
$
<br>
Instead of initializing the set component by rewriting all items, we can extract them from the previously created dictionary v. Calling __v.keys()__ returns all the keys (our items) in dictionary v. That saves some typing:

In [ ]:
mo.I = Set(initialize=v.keys())

In [ ]:
mo.I.pprint()

### Step 4: Based on the index set, define the decision variables
$
\begin{equation*}
X_i \qquad i \in I, \: X \in \{0,1\}\\
\end{equation*}
$

In [ ]:
mo.X = Var(mo.I, within=Binary, initialize=0)

In [ ]:
mo.X.pprint()

### Step 5: Specify the objective
$
\begin{equation*}
MAX \: \sum\limits_{i \in I}v_iX_i \\
\end{equation*}
$

In [ ]:
mo.obj = Objective(sense=maximize,
                    expr=sum(v[i]*mo.X[i] for i in mo.I))

In [ ]:
mo.obj.pprint()

### Step 6: Specify the constraints

#### Constraint: Maximum weight
$
\begin{equation*}
\sum\limits_{i \in I}w_iX_i \leq w_{max}\\
\end{equation*}
$

In [ ]:
mo.c_max_weight = Constraint(expr=sum(w[i]*mo.X[i] for i in mo.I) <= w_max)

In [ ]:
mo.c_max_weight.pprint()

### Step 7: Decide on a suitable solver depending on your problem and solve it

In [ ]:
with open('logs/opti_model.txt', 'w') as f:
    mo.pprint(ostream=f)

In [ ]:
print('--- start solver ---')
solver = SolverFactory('cbc', executable=solver_path)
solver.solve(mo, tee=True, logfile='logs/solver_log.txt')
print('--- finished ---')

### Step 8: Process the results

In [ ]:
print('Total value: ' + str(value(mo.obj)) + ' [$]')
print('-'*20)

for k,v in mo.X.items():
    print(str(k) + ': ' + str(int(value(v))))

Just out of curiosity, let's check that the maximum weight constraint actually is satisfied:

In [ ]:
total_weight = 0
for k,v in mo.X.items():
    if value(v) > 0:
        total_weight += w[k]
print('Total knapsack weight: ' + str(total_weight) + ' [g]')
print('Maximum weight: ' + str(w_max) + ' [g]')

#### Looks good!